In [2]:
import os, subprocess
print("dev nvidia0:", os.path.exists("/dev/nvidia0"))
print("nvidia-smi -L:\n", subprocess.getoutput("nvidia-smi -L"))

dev nvidia0: True
nvidia-smi -L:
 GPU 0: Tesla T4 (UUID: GPU-ef376eeb-aeb8-d557-da7c-ff7125931a0f)


In [3]:
%%bash
set -e
curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
ls -l ./bin/micromamba

bin/micromamba
-rwxrwxr-x 1 root root 17546920 Feb 11 14:12 ./bin/micromamba


In [4]:
%%bash
set -e
./bin/micromamba create -y -n omm83 -c conda-forge python=3.10 "openmm=8.3.*" mdtraj parmed ambertools matplotlib



Transaction

  Prefix: /root/.local/share/mamba/envs/omm83

  Updating specs:

   - python=3.10
   - openmm=8.3
   - mdtraj
   - parmed
   - ambertools
   - matplotlib


  Package                           Version  Build                              Channel          Size
───────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────────────────────────────

  + _openmp_mutex                       4.5  7_kmp_llvm                         conda-forge       8kB
  + alsa-lib                       1.2.15.3  hb03c661_0                         conda-forge     585kB
  + ambertools                         24.8  cuda_None_nompi_py310h834fefc_101  conda-forge     101MB
  + arpack                            3.9.1  nompi_hf03ea27_102                 conda-forge     130kB
  + blosc                            1.21.6  he440d0b_1                         conda-forge      4

In [5]:
%%bash
./bin/micromamba run -n omm83 python -c "from openmm import Platform; import openmm; print('OpenMM', openmm.__version__); print('Platforms:'); [print(' ', i, Platform.getPlatform(i).getName()) for i in range(Platform.getNumPlatforms())]"

OpenMM 8.3.1
Platforms:
  0 Reference
  1 CPU
  2 CUDA
  3 OpenCL


In [6]:
from google.colab import files
import os

required = ["fiber_21mol_TOL.prmtop", "fiber_21mol_TOL.inpcrd"]
print("Upload these files:")
for f in required:
    print(" -", f)

uploaded = files.upload()

print("\nFiles in /content:")
for f in required:
    print(f, "->", "FOUND" if os.path.exists("/content/" + f) else "MISSING")

Upload these files:
 - fiber_21mol_TOL.prmtop
 - fiber_21mol_TOL.inpcrd


Saving fiber_21mol_TOL.inpcrd to fiber_21mol_TOL.inpcrd
Saving fiber_21mol_TOL.prmtop to fiber_21mol_TOL.prmtop

Files in /content:
fiber_21mol_TOL.prmtop -> FOUND
fiber_21mol_TOL.inpcrd -> FOUND


In [7]:
%%bash
set -e
cat > /content/smoke_test_cuda.py << 'PY'
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PME, HBonds, StateDataReporter
from openmm import LangevinIntegrator, Platform
from openmm.unit import kelvin, picosecond, nanometer
import sys

prmtop = AmberPrmtopFile("/content/fiber_21mol_TOL.prmtop")
inpcrd = AmberInpcrdFile("/content/fiber_21mol_TOL.inpcrd")

system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinIntegrator(323.15*kelvin, 0.3/picosecond, 0.002*picosecond)

platform = Platform.getPlatformByName("CUDA")
props = {"Precision": "mixed"}

sim = Simulation(prmtop.topology, system, integrator, platform, props)
sim.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    sim.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

print("Using platform:", sim.context.getPlatform().getName())
sim.minimizeEnergy()
sim.context.setVelocitiesToTemperature(323.15*kelvin)
sim.reporters.append(StateDataReporter(sys.stdout, 500, step=True, temperature=True, potentialEnergy=True))
sim.step(2000)
print("OK: 2000 steps completed.")
PY

./bin/micromamba run -n omm83 python /content/smoke_test_cuda.py

Using platform: CUDA
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
500,121432.21774923988,210.32251913279202
1000,121671.21816830577,246.46719517666864
1500,122040.40389266373,270.03912857205233
2000,122424.57053180947,283.3245081915807
OK: 2000 steps completed.


In [8]:
%%bash
set -e
cat > /content/run_ramp_cuda.py << 'PY'
from openmm.app import (
    AmberPrmtopFile, AmberInpcrdFile, Simulation,
    PME, HBonds, DCDReporter, StateDataReporter, PDBFile
)
from openmm import LangevinIntegrator, Platform
from openmm.unit import kelvin, picosecond, nanometer

prmtop = AmberPrmtopFile("/content/fiber_21mol_TOL.prmtop")
inpcrd = AmberInpcrdFile("/content/fiber_21mol_TOL.inpcrd")

system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinIntegrator(323.15*kelvin, 0.3/picosecond, 0.002*picosecond)

platform = Platform.getPlatformByName("CUDA")
props = {"Precision": "mixed"}

sim = Simulation(prmtop.topology, system, integrator, platform, props)
sim.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    sim.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

print("Using platform:", sim.context.getPlatform().getName())

sim.minimizeEnergy()
sim.context.setVelocitiesToTemperature(323.15*kelvin)

print("Equilibration: 50 ps @ 323.15 K")
sim.step(25000)

sim.reporters.append(DCDReporter("/content/fiber_ramp_50C_to_10C.dcd", 500))
sim.reporters.append(StateDataReporter(
    "/content/fiber_ramp_50C_to_10C.log", 500,
    step=True, temperature=True, potentialEnergy=True, density=True, speed=True
))

n_steps = 250000
start_temp = 323.15
end_temp = 283.15

block_size = 250
n_blocks = n_steps // block_size
assert n_blocks * block_size == n_steps

print(f"Ramp: {start_temp:.2f} K -> {end_temp:.2f} K | steps={n_steps} | blocks={n_blocks} | block={block_size}")

for b in range(n_blocks):
    frac = b/(n_blocks-1) if n_blocks > 1 else 1.0
    T = start_temp + frac*(end_temp-start_temp)
    integrator.setTemperature(T*kelvin)
    sim.step(block_size)

state = sim.context.getState(getPositions=True)
pos = state.getPositions()

with open("/content/fiber_10C_final.pdb", "w") as f:
    PDBFile.writeFile(prmtop.topology, pos, f)

print("DONE. Outputs written to /content.")
PY

./bin/micromamba run -n omm83 python /content/run_ramp_cuda.py
ls -lh /content/fiber_ramp_50C_to_10C.dcd /content/fiber_ramp_50C_to_10C.log /content/fiber_10C_final.pdb | cat

Using platform: CUDA
Equilibration: 50 ps @ 323.15 K
Ramp: 323.15 K -> 283.15 K | steps=250000 | blocks=1000 | block=250
DONE. Outputs written to /content.
-rw-r--r-- 1 root root 141K Feb 26 00:09 /content/fiber_10C_final.pdb
-rw-r--r-- 1 root root 8.0M Feb 26 00:09 /content/fiber_ramp_50C_to_10C.dcd
-rw-r--r-- 1 root root  36K Feb 26 00:09 /content/fiber_ramp_50C_to_10C.log


In [9]:
import os
for f in ["fiber_ramp_50C_to_10C.dcd", "fiber_ramp_50C_to_10C.log", "fiber_10C_final.pdb"]:
    print(f, "->", "FOUND" if os.path.exists("/content/" + f) else "MISSING")

fiber_ramp_50C_to_10C.dcd -> FOUND
fiber_ramp_50C_to_10C.log -> FOUND
fiber_10C_final.pdb -> FOUND


In [10]:
%%bash
set -e
cat > /content/analysis_common.py << 'PY'
import os
os.environ.pop("MPLBACKEND", None)
os.environ["MPLBACKEND"] = "Agg"

import numpy as np
import mdtraj as md

TRAJ_PATH = "/content/fiber_ramp_50C_to_10C.dcd"
TOP_PATH  = "/content/fiber_21mol_TOL.prmtop"

FRAME_TEMP_MAP = {
    0: 50,
    125: 40,
    249: 30,
    374: 20,
    399: 18,
    412: 17,
    424: 16,
    437: 15,
    449: 14,
    474: 12,
    499: 10
}

MONOMER_ATOM_COUNT = 52
FIBER_MONOMER_COUNT = 21

ATOM_1_REL = 20
ATOM_2_REL = 19
ATOM_3_REL = 71 - MONOMER_ATOM_COUNT
ATOM_4_REL = 72 - MONOMER_ATOM_COUNT

ATOM_SINGLE = (20, 19, 71, 72)
ATOM_N_IN_MONOMER = 3

def load_traj():
    return md.load(TRAJ_PATH, top=TOP_PATH)

def compute_dihedral(p0, p1, p2, p3):
    b0 = -1.0 * (p1 - p0)
    b1 = p2 - p1
    b2 = p3 - p2
    b1 /= np.linalg.norm(b1)
    v = b0 - np.dot(b0, b1) * b1
    w = b2 - np.dot(b2, b1) * b1
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

def wrap180(angle_deg):
    if angle_deg > 180:
        angle_deg -= 360
    elif angle_deg < -180:
        angle_deg += 360
    return angle_deg

def pca_first_axis(X):
    Xc = X - X.mean(axis=0, keepdims=True)
    _, _, vt = np.linalg.svd(Xc, full_matrices=False)
    axis = vt[0]
    axis /= np.linalg.norm(axis)
    return axis
PY

In [11]:
%%bash
set -e
cat > /content/analysis_dihedrals.py << 'PY'
import matplotlib.pyplot as plt
import numpy as np
from analysis_common import (
    load_traj, FRAME_TEMP_MAP, compute_dihedral, wrap180,
    MONOMER_ATOM_COUNT, FIBER_MONOMER_COUNT,
    ATOM_1_REL, ATOM_2_REL, ATOM_3_REL, ATOM_4_REL, ATOM_SINGLE
)

traj = load_traj()

# (A) Segmental
a1, a2, a3, a4 = ATOM_SINGLE
temps = []
dihed = []
for frame, temp in FRAME_TEMP_MAP.items():
    p = traj.xyz[frame]
    d = wrap180(compute_dihedral(p[a1], p[a2], p[a3], p[a4]))
    temps.append(temp)
    dihed.append(d)

plt.figure(figsize=(3.5, 2.5))
plt.plot(temps, dihed, marker="o", label="Segmental torsion angle (N–N+1)")
plt.axhline(0, linestyle="--")
plt.axvline(16, linestyle="--", label="16°C threshold")
plt.xlim(55, 2)
plt.ylim(-10, 100)
plt.xlabel("Temperature (°C)", fontsize=10)
plt.ylabel("Torsion Angle (°)", fontsize=10)
plt.xticks(fontsize=9); plt.yticks(fontsize=9)
plt.legend(fontsize=9, loc="best")
plt.grid(True)
plt.tight_layout()
plt.savefig("/content/dihedral_vs_temperature_map.png", dpi=300)
plt.close()

# (B) Fiber-average (20 interfaces), descending temp order
temps_sorted = sorted(FRAME_TEMP_MAP.items(), key=lambda x: -x[1])
temps_avg = []
avg_vals = []
for frame, temp in temps_sorted:
    p = traj.xyz[frame]
    vals = []
    for i in range(FIBER_MONOMER_COUNT - 1):
        base = i * MONOMER_ATOM_COUNT
        nxt  = (i + 1) * MONOMER_ATOM_COUNT
        d = compute_dihedral(
            p[base + ATOM_1_REL],
            p[base + ATOM_2_REL],
            p[nxt  + ATOM_3_REL],
            p[nxt  + ATOM_4_REL],
        )
        vals.append(wrap180(d))
    temps_avg.append(temp)
    avg_vals.append(float(np.mean(vals)))

plt.figure(figsize=(3.5, 2.5))
plt.plot(temps_avg, avg_vals, marker="o", label="Avg. torsion angle")
plt.axhline(0, linestyle="--")
plt.axvline(16, linestyle="--", label="16°C threshold")
plt.xlim(55, 5)
plt.ylim(-10, 100)
plt.xlabel("Temperature (°C)", fontsize=10)
plt.ylabel("Avg. Torsion Angle (°)", fontsize=10)
plt.xticks(fontsize=9); plt.yticks(fontsize=9)
plt.legend(fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.savefig("/content/average_dihedral_vs_temperature_map.png", dpi=300)
plt.close()

print("Wrote dihedral plots to /content.")
PY

MPLBACKEND=Agg ./bin/micromamba run -n omm83 python /content/analysis_dihedrals.py

Wrote dihedral plots to /content.


In [12]:
%%bash
set -e
cat > /content/analysis_dihedral_heatmap.py << 'PY'
import matplotlib.pyplot as plt
import numpy as np
from analysis_common import (
    load_traj, FRAME_TEMP_MAP, compute_dihedral, wrap180,
    MONOMER_ATOM_COUNT, FIBER_MONOMER_COUNT,
    ATOM_1_REL, ATOM_2_REL, ATOM_3_REL, ATOM_4_REL
)

traj = load_traj()
temps_sorted = sorted(FRAME_TEMP_MAP.items(), key=lambda x: -x[1])

heat = []
for frame, temp in temps_sorted:
    p = traj.xyz[frame]
    row = []
    for i in range(FIBER_MONOMER_COUNT - 1):
        base = i * MONOMER_ATOM_COUNT
        nxt  = (i + 1) * MONOMER_ATOM_COUNT
        d = compute_dihedral(
            p[base + ATOM_1_REL],
            p[base + ATOM_2_REL],
            p[nxt  + ATOM_3_REL],
            p[nxt  + ATOM_4_REL],
        )
        row.append(wrap180(d))
    heat.append(row)

heat = np.array(heat)

plt.figure(figsize=(8, 4))
im = plt.imshow(
    heat, aspect="auto", cmap="coolwarm", vmin=-120, vmax=120,
    extent=[0, FIBER_MONOMER_COUNT - 1, temps_sorted[-1][1], temps_sorted[0][1]]
)
cbar = plt.colorbar(im)
cbar.set_label("Dihedral angle (°) signed", fontsize=14)
cbar.ax.tick_params(labelsize=13)
cbar.set_ticks(np.arange(-120, 121, 40))

plt.axhline(y=16, color="black", linestyle="--", linewidth=1.5, label="16 °C threshold")
plt.xlabel("Fiber segment index (Monomer N to N+1)", fontsize=14)
plt.ylabel("Temperature (°C)", fontsize=14)
plt.xticks(ticks=np.arange(0, FIBER_MONOMER_COUNT, 1),
           labels=[str(i) for i in range(FIBER_MONOMER_COUNT)], fontsize=12)
plt.yticks(fontsize=14)
plt.legend(fontsize=14)
plt.tight_layout()
plt.savefig("/content/fiber_segment_dihedral_heatmap.png", dpi=300)
plt.close()

print("Wrote: /content/fiber_segment_dihedral_heatmap.png")
PY

MPLBACKEND=Agg ./bin/micromamba run -n omm83 python /content/analysis_dihedral_heatmap.py

Wrote: /content/fiber_segment_dihedral_heatmap.png


In [13]:
%%bash
set -e
cat > /content/analysis_stacking_pca.py << 'PY'
import matplotlib.pyplot as plt
import numpy as np
from analysis_common import (
    load_traj, FRAME_TEMP_MAP, pca_first_axis,
    MONOMER_ATOM_COUNT, FIBER_MONOMER_COUNT, ATOM_N_IN_MONOMER
)

traj = load_traj()
all_dist = []
temps = []

for frame, temp in FRAME_TEMP_MAP.items():
    Npos = []
    for i in range(FIBER_MONOMER_COUNT):
        base = i * MONOMER_ATOM_COUNT
        Npos.append(traj.xyz[frame, base + ATOM_N_IN_MONOMER, :])
    Npos = np.array(Npos)

    axis = pca_first_axis(Npos)
    proj = np.dot(Npos, axis)
    dist = np.abs(proj[1:] - proj[:-1])

    all_dist.append(dist)
    temps.append(temp)

data = np.array(all_dist)
median = np.median(data, axis=1)
q1 = np.percentile(data, 25, axis=1)
q3 = np.percentile(data, 75, axis=1)

plt.figure(figsize=(3.5, 2.5))
plt.plot(temps, median, marker="D", label="Median stacking distance")
plt.fill_between(temps, q1, q3, alpha=0.3, label="Interquartile range")
plt.xlim(55, 5)
plt.ylim(0.275, 0.425)
plt.xlabel("Temperature (°C)", fontsize=10)
plt.ylabel("π–π stacking distance (nm)", fontsize=10)
plt.xticks(fontsize=9); plt.yticks(fontsize=9)
plt.legend(fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.savefig("/content/pi_pi_stacking_distance_vs_temperature.png", dpi=300)
plt.close()

print("Wrote: /content/pi_pi_stacking_distance_vs_temperature.png")
PY

MPLBACKEND=Agg ./bin/micromamba run -n omm83 python /content/analysis_stacking_pca.py

Wrote: /content/pi_pi_stacking_distance_vs_temperature.png


In [14]:
%%bash
set -e
cat > /content/analysis_lateral_shift.py << 'PY'
import matplotlib.pyplot as plt
import numpy as np
from analysis_common import (
    load_traj, FRAME_TEMP_MAP, pca_first_axis,
    MONOMER_ATOM_COUNT, FIBER_MONOMER_COUNT, ATOM_N_IN_MONOMER
)

traj = load_traj()
all_lat = []
temps = []

for frame, temp in FRAME_TEMP_MAP.items():
    Npos = []
    for i in range(FIBER_MONOMER_COUNT):
        base = i * MONOMER_ATOM_COUNT
        Npos.append(traj.xyz[frame, base + ATOM_N_IN_MONOMER, :])
    Npos = np.array(Npos)

    axis = pca_first_axis(Npos)

    shifts = []
    for i in range(FIBER_MONOMER_COUNT - 1):
        delta = Npos[i+1] - Npos[i]
        delta_proj = np.dot(delta, axis) * axis
        lateral_vec = delta - delta_proj
        shifts.append(np.linalg.norm(lateral_vec))
    all_lat.append(shifts)
    temps.append(temp)

data = np.array(all_lat)
median = np.median(data, axis=1)
q1 = np.percentile(data, 25, axis=1)
q3 = np.percentile(data, 75, axis=1)

plt.figure(figsize=(3.5, 2.5))
plt.boxplot(data.T, positions=temps, widths=1.2, patch_artist=True,
            medianprops=dict(color="black"), whiskerprops=dict(color="grey"))
plt.plot(temps, median, marker="D", label="Median lateral shift")
plt.fill_between(temps, q1, q3, alpha=0.3, label="Interquartile range")

plt.xlim(55, 5)
plt.ylim(0, 0.8)
plt.xticks([50, 40, 30, 20, 16, 10], ["50", "40", "30", "20", "16", "10"], fontsize=9)
plt.xlabel("Temperature (°C)", fontsize=10)
plt.ylabel("Lateral shift (nm)", fontsize=10)
plt.yticks(fontsize=9)
plt.legend(fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.savefig("/content/lateral_shift_vs_temperature.png", dpi=300)
plt.close()

print("Wrote: /content/lateral_shift_vs_temperature.png")
PY

MPLBACKEND=Agg ./bin/micromamba run -n omm83 python /content/analysis_lateral_shift.py

Wrote: /content/lateral_shift_vs_temperature.png


In [15]:
%%bash
set -e
cat > /content/make_fiber_dcd_top.py << 'PY'
import mdtraj as md
import numpy as np

traj = md.load("/content/fiber_ramp_50C_to_10C.dcd", top="/content/fiber_21mol_TOL.prmtop")

fiber_atoms = 21 * 52
fiber = traj.atom_slice(np.arange(fiber_atoms))

fiber.save_dcd("/content/fiber_only.dcd")
fiber[0].save_pdb("/content/fiber_only.pdb")

print("Wrote /content/fiber_only.dcd and /content/fiber_only.pdb")
print("Frames:", fiber.n_frames, "Atoms:", fiber.n_atoms)
PY

./bin/micromamba run -n omm83 python /content/make_fiber_dcd_top.py
ls -lh /content/fiber_only.dcd /content/fiber_only.pdb | cat

Wrote /content/fiber_only.dcd and /content/fiber_only.pdb
Frames: 500 Atoms: 1092
-rw-r--r-- 1 root root 6.3M Feb 26 00:09 /content/fiber_only.dcd
-rw-r--r-- 1 root root 112K Feb 26 00:09 /content/fiber_only.pdb


In [21]:
%%bash
set -e
cat > /content/benchmark_openmm.py << 'PY'
import time
import platform as py_platform
from dataclasses import dataclass

from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PME, HBonds
from openmm import LangevinIntegrator, Platform
from openmm.unit import kelvin, picosecond, nanometer

@dataclass
class BenchResult:
    platform: str
    precision: str
    steps: int
    dt_ps: float
    wall_s: float
    ns_per_day: float

def build_sim(prmtop_path: str, inpcrd_path: str, platform_name: str, precision: str):
    prmtop = AmberPrmtopFile(prmtop_path)
    inpcrd = AmberInpcrdFile(inpcrd_path)

    system = prmtop.createSystem(
        nonbondedMethod=PME,
        nonbondedCutoff=1.0 * nanometer,
        constraints=HBonds,
    )

    integrator = LangevinIntegrator(323.15 * kelvin, 0.3 / picosecond, 0.002 * picosecond)

    if platform_name == "CUDA":
        platform = Platform.getPlatformByName("CUDA")
        props = {"Precision": precision}
        sim = Simulation(prmtop.topology, system, integrator, platform, props)
    else:
        platform = Platform.getPlatformByName(platform_name)
        sim = Simulation(prmtop.topology, system, integrator, platform)

    sim.context.setPositions(inpcrd.positions)
    if inpcrd.boxVectors is not None:
        sim.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

    return sim

def run_bench(sim: Simulation, steps_warmup: int, steps_bench: int, dt_ps: float, platform_name: str, precision: str):
    # Warmup: JIT compile / kernel init / cache fill
    sim.minimizeEnergy()
    sim.context.setVelocitiesToTemperature(323.15 * kelvin)
    sim.step(steps_warmup)

    t0 = time.perf_counter()
    sim.step(steps_bench)
    t1 = time.perf_counter()

    wall = t1 - t0
    sim_ns = (steps_bench * dt_ps) / 1000.0  # ps -> ns
    ns_per_day = sim_ns / (wall / 86400.0)

    return BenchResult(
        platform=platform_name,
        precision=precision,
        steps=steps_bench,
        dt_ps=dt_ps,
        wall_s=wall,
        ns_per_day=ns_per_day,
    )

def main():
    prmtop_path = "/content/fiber_21mol_TOL.prmtop"
    inpcrd_path = "/content/fiber_21mol_TOL.inpcrd"

    dt_ps = 0.002  # 2 fs
    steps_warmup = 2000
    steps_bench  = 20000  # keep small enough for Colab, large enough for stable timing

    print("System:", prmtop_path, inpcrd_path)
    print("Python:", py_platform.python_version())
    print("OS:", py_platform.platform())

    # CPU baseline
    sim_cpu = build_sim(prmtop_path, inpcrd_path, platform_name="CPU", precision="n/a")
    r_cpu = run_bench(sim_cpu, steps_warmup, steps_bench, dt_ps, platform_name="CPU", precision="n/a")
    print(f"[CPU]  wall={r_cpu.wall_s:.2f} s | ns/day={r_cpu.ns_per_day:.2f}")

    # CUDA mixed precision
    sim_cuda = build_sim(prmtop_path, inpcrd_path, platform_name="CUDA", precision="mixed")
    r_cuda = run_bench(sim_cuda, steps_warmup, steps_bench, dt_ps, platform_name="CUDA", precision="mixed")
    print(f"[CUDA mixed] wall={r_cuda.wall_s:.2f} s | ns/day={r_cuda.ns_per_day:.2f}")

    speedup = r_cuda.ns_per_day / r_cpu.ns_per_day if r_cpu.ns_per_day > 0 else float("inf")
    print(f"Speedup (CUDA/CPU): {speedup:.2f}x")

if __name__ == '__main__':
    main()
PY

In [22]:
%%bash
set -e
./bin/micromamba run -n omm83 python /content/benchmark_openmm.py

System: /content/fiber_21mol_TOL.prmtop /content/fiber_21mol_TOL.inpcrd
Python: 3.10.19
OS: Linux-6.6.113+-x86_64-with-glibc2.35
[CPU]  wall=104.23 s | ns/day=33.16
[CUDA mixed] wall=2.56 s | ns/day=1348.09
Speedup (CUDA/CPU): 40.66x


In [17]:
%%bash
set -e
cd /content
REPO_DIR="colab_cuda_openmm_fiber"
mkdir -p "$REPO_DIR"
cd "$REPO_DIR"
pwd
ls -lah

/content/colab_cuda_openmm_fiber
total 8.0K
drwxr-xr-x 2 root root 4.0K Feb 26 00:09 .
drwxr-xr-x 1 root root 4.0K Feb 26 00:09 ..


In [23]:
%%bash
set -e
cd /content/colab_cuda_openmm_fiber
mkdir -p scripts environment notebooks

cp -f /content/run_ramp_cuda.py scripts/ 2>/dev/null || true
cp -f /content/smoke_test_cuda.py scripts/ 2>/dev/null || true
cp -f /content/analysis_common.py scripts/ 2>/dev/null || true
cp -f /content/analysis_dihedrals.py scripts/ 2>/dev/null || true
cp -f /content/analysis_dihedral_heatmap.py scripts/ 2>/dev/null || true
cp -f /content/analysis_stacking_pca.py scripts/ 2>/dev/null || true
cp -f /content/analysis_lateral_shift.py scripts/ 2>/dev/null || true
cp -f /content/make_fiber_dcd_top.py scripts/ 2>/dev/null || true
cp -f /content/benchmark_openmm.py scripts/ 2>/dev/null || true

ls -lah scripts

total 44K
drwxr-xr-x 2 root root 4.0K Feb 26 00:25 .
drwxr-xr-x 5 root root 4.0K Feb 26 00:10 ..
-rw-r--r-- 1 root root 1.3K Feb 26 00:25 analysis_common.py
-rw-r--r-- 1 root root 1.7K Feb 26 00:25 analysis_dihedral_heatmap.py
-rw-r--r-- 1 root root 2.2K Feb 26 00:25 analysis_dihedrals.py
-rw-r--r-- 1 root root 1.7K Feb 26 00:25 analysis_lateral_shift.py
-rw-r--r-- 1 root root 1.3K Feb 26 00:25 analysis_stacking_pca.py
-rw-r--r-- 1 root root 3.2K Feb 26 00:25 benchmark_openmm.py
-rw-r--r-- 1 root root  417 Feb 26 00:25 make_fiber_dcd_top.py
-rw-r--r-- 1 root root 1.9K Feb 26 00:25 run_ramp_cuda.py
-rw-r--r-- 1 root root 1.1K Feb 26 00:25 smoke_test_cuda.py


In [24]:
%%bash
set -e
cd /content/colab_cuda_openmm_fiber

cat > environment/env.yml << 'YML'
name: omm83
channels:
  - conda-forge
dependencies:
  - python=3.10
  - openmm=8.3.*
  - mdtraj
  - parmed
  - ambertools
  - matplotlib
YML

cat > .gitignore << 'TXT'
*.dcd
*.xtc
*.trr
*.nc
*.log
*.pdb
*.prmtop
*.inpcrd
__pycache__/
.ipynb_checkpoints/
TXT

cat > README.md << 'MD'
# colab_cuda_openmm_fiber

CUDA-accelerated OpenMM workflow (Colab) for a GAFF-parametrized supramolecular fiber in explicit toluene.

## What is included
- `scripts/run_ramp_cuda.py`: minimize → 50 ps @ 323.15 K → 500 ps ramp (323.15→283.15 K) on CUDA
- `scripts/analysis_common.py`: shared analysis utilities
- `scripts/analysis_*`: analysis plots (dihedrals, heatmap, stacking distance, lateral shift)
- `scripts/make_fiber_dcd_top.py`: export fiber-only DCD+PDB for visualization
- `environment/env.yml`: reproducible environment (OpenMM pinned to 8.3.* to avoid PTX mismatch on Colab T4)

Large outputs (DCD, prmtop/inpcrd, etc.) are excluded via `.gitignore`.
MD

ls -lah

total 28K
drwxr-xr-x 5 root root 4.0K Feb 26 00:10 .
drwxr-xr-x 1 root root 4.0K Feb 26 00:19 ..
drwxr-xr-x 2 root root 4.0K Feb 26 00:10 environment
-rw-r--r-- 1 root root   86 Feb 26 00:25 .gitignore
drwxr-xr-x 2 root root 4.0K Feb 26 00:09 notebooks
-rw-r--r-- 1 root root  672 Feb 26 00:25 README.md
drwxr-xr-x 2 root root 4.0K Feb 26 00:25 scripts


In [25]:
%%bash
set -e
cd /content
zip -r colab_cuda_openmm_fiber.zip colab_cuda_openmm_fiber
ls -lh colab_cuda_openmm_fiber.zip

updating: colab_cuda_openmm_fiber/ (stored 0%)
updating: colab_cuda_openmm_fiber/.gitignore (deflated 19%)
updating: colab_cuda_openmm_fiber/environment/ (stored 0%)
updating: colab_cuda_openmm_fiber/environment/env.yml (deflated 23%)
updating: colab_cuda_openmm_fiber/scripts/ (stored 0%)
updating: colab_cuda_openmm_fiber/scripts/analysis_dihedrals.py (deflated 63%)
updating: colab_cuda_openmm_fiber/scripts/make_fiber_dcd_top.py (deflated 46%)
updating: colab_cuda_openmm_fiber/scripts/analysis_stacking_pca.py (deflated 51%)
updating: colab_cuda_openmm_fiber/scripts/run_ramp_cuda.py (deflated 55%)
updating: colab_cuda_openmm_fiber/scripts/analysis_common.py (deflated 49%)
updating: colab_cuda_openmm_fiber/scripts/smoke_test_cuda.py (deflated 50%)
updating: colab_cuda_openmm_fiber/scripts/analysis_lateral_shift.py (deflated 53%)
updating: colab_cuda_openmm_fiber/scripts/analysis_dihedral_heatmap.py (deflated 54%)
updating: colab_cuda_openmm_fiber/README.md (deflated 36%)
updating: colab_